<a href="https://colab.research.google.com/github/lakshayydua/WeblogChallenge/blob/master/WebLogAnalytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# To set up Pyspark environment in Colab notebook
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
import os
import sys
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("WebLogAnalytics").master("local[*]").getOrCreate()

In [4]:
# spark env configuration
sc = spark.sparkContext
print("LocalProperty: ", sc.getLocalProperty)
print("defaultParallelism: ", sc.defaultParallelism)
print("ExecutorMemoryStatus: ", sc._jsc.sc().getExecutorMemoryStatus().size())

('LocalProperty: ', <bound method SparkContext.getLocalProperty of <SparkContext master=local[*] appName=WebLogAnalytics>>)
('defaultParallelism: ', 2)
('ExecutorMemoryStatus: ', 1)


In [0]:
from pyspark.sql import SparkSession, types, Window
from pyspark.sql.functions import udf, pandas_udf, PandasUDFType
from pyspark.sql.functions import split, explode, struct, arrays_zip, col
from datetime import datetime, date, timedelta

In [0]:
# creating a pyspark.sql.dataframe.DataFrame object
explicit_schema = types.StructType([
    types.StructField("timestamp", types.TimestampType(), True),
    types.StructField("elb", types.StringType(), True),
    types.StructField("client_port", types.StringType(), True),
    types.StructField("backend_port", types.StringType(), True),
    types.StructField("request_processing_time", types.DoubleType(), True),
    types.StructField("backend_processing_time", types.DoubleType(), True),
    types.StructField("response_processing_time", types.DoubleType(), True),
    types.StructField("elb_status_code", types.IntegerType(), True),
    types.StructField("backend_status_code", types.IntegerType(), True),
    types.StructField("received_bytes", types.DoubleType(), True),
    types.StructField("sent_bytes", types.DoubleType(), True),
    types.StructField("request", types.StringType(), True),
    types.StructField("user_agent", types.StringType(), True),
    types.StructField("ssl_cipher", types.StringType(), True),
    types.StructField("ssl_protocol", types.StringType(), True)
])

log_df = spark.read.format("csv") \
              .schema(explicit_schema) \
              .option("header", "false") \
              .option("delimiter", " ") \
              .load("/content/data/")

In [7]:
log_df.show()
log_df.createOrReplaceTempView("log_df")

+--------------------+----------------+--------------------+-------------+-----------------------+-----------------------+------------------------+---------------+-------------------+--------------+----------+--------------------+--------------------+--------------------+------------+
|           timestamp|             elb|         client_port| backend_port|request_processing_time|backend_processing_time|response_processing_time|elb_status_code|backend_status_code|received_bytes|sent_bytes|             request|          user_agent|          ssl_cipher|ssl_protocol|
+--------------------+----------------+--------------------+-------------+-----------------------+-----------------------+------------------------+---------------+-------------------+--------------+----------+--------------------+--------------------+--------------------+------------+
|2015-07-22 09:00:...|marketpalce-shop|123.242.248.130:5...|10.0.6.158:80|                 2.2E-5|               0.026109|                  2.

## **Processing & Analytical Goals**

To uniquely identify a client or user, a derived field named "visitor_id" has been utilized. 


> visitor_id is derived from following fields present in the log file:

1.   client_port
2.   user_agent



In [8]:
log_df_summary = spark.sql("""
                            SELECT COUNT(client_port) total_records_count, 
                            COUNT(DISTINCT CONCAT(client_port, '_', user_agent)) 
                            unique_visitor_count, 
                            MIN(timestamp) min_timestamp, 
                            MAX(timestamp) max_timestamp
                            FROM log_df
                            """)

log_df_summary.show(truncate=False)

+-------------------+--------------------+-----------------------+-----------------------+
|total_records_count|unique_visitor_count|min_timestamp          |max_timestamp          |
+-------------------+--------------------+-----------------------+-----------------------+
|1158500            |413200              |2015-07-22 02:40:16.121|2015-07-22 21:27:03.632|
+-------------------+--------------------+-----------------------+-----------------------+



In [9]:
derived_df = spark.sql("""
                      SELECT CONCAT(client_port, '_', user_agent) visitor_id,
                      request,
                      timestamp
                      FROM log_df
                      """)

# repartition on visitor_id to speed up aggregation
# order by increasing order of timestamps required by sessionizing algorithm
derived_df = derived_df.repartition("visitor_id").orderBy("timestamp")
derived_df.createOrReplaceTempView("derived_df")
derived_df.show()

+--------------------+--------------------+--------------------+
|          visitor_id|             request|           timestamp|
+--------------------+--------------------+--------------------+
|119.81.61.166:460...|GET https://paytm...|2015-07-22 02:40:...|
|1.39.62.163:34315...|GET https://paytm...|2015-07-22 02:40:...|
|1.39.60.241:50549...|GET https://paytm...|2015-07-22 02:40:...|
|119.81.61.166:449...|GET https://paytm...|2015-07-22 02:40:...|
|117.221.185.222:5...|GET https://paytm...|2015-07-22 02:40:...|
|119.81.61.166:458...|GET https://paytm...|2015-07-22 02:40:...|
|119.81.61.166:456...|GET https://paytm...|2015-07-22 02:40:...|
|119.81.61.166:460...|GET https://paytm...|2015-07-22 02:40:...|
|27.6.176.251:4950...|POST https://payt...|2015-07-22 02:40:...|
|27.107.95.49:1266...|GET https://paytm...|2015-07-22 02:40:...|
|182.71.61.18:3977...|GET https://paytm...|2015-07-22 02:40:...|
|115.99.105.216:50...|GET https://paytm...|2015-07-22 02:40:...|
|54.169.106.125:45...|GET

In [10]:
# aggregating requests and timestamp data for every visitor_id
agg_timestamp_df = spark.sql("""
                        SELECT visitor_id,
                        COLLECT_LIST(request) request_list,
                        COLLECT_LIST(timestamp) timestamp_list,
                        COUNT(*) timestamp_count
                        FROM derived_df
                        GROUP BY visitor_id
                        """)

agg_timestamp_df.show(truncate=False)
agg_timestamp_df.createOrReplaceTempView("agg_timestamp_df")

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## **1. Sessionize the web log by IP. Sessionize = aggregrate all page hits by visitor/IP during a session**

In [0]:
# Inactivity threshold is set to 15 minutes or 15*60*1000 milliseconds
THRESHOLD = 15*60*1000
    
def sessionize_log(timestamp_list):
  """
    This UDF function expects timestamp_list in increasing order of timestamps
    for a specific visitor_id. A session becomes inactive after 15 minutes of 
    inactivity and thus any new request opens a new session. New session is 
    shown by increment old session_id by 1.

    Arguments:
        timestamp_list {list} -- list of timestamps in increasing order
  """
  
  last_active = timestamp_list[0]
  session_id = 1
  session_list = [session_id]
  
  # early exit for a visitor with only one timestamp
  if len(timestamp_list) == 1:
    return session_list

  for timestamp in timestamp_list[1:]:
    # if the (current timestamp - last recorded timestamp) < 15 mins then 
    # old session remains active
    if timestamp < last_active + timedelta(milliseconds=THRESHOLD):
      last_active = timestamp
      session_list.append(session_id)
    else:
      last_active = timestamp
      # increment session_id if old_session became inactive
      session_id = session_id + 1
      session_list.append(session_id)

  return session_list

sessionize_log_udf = udf(sessionize_log, types.ArrayType(types.IntegerType()))

In [0]:
agg_sessionized_df = agg_timestamp_df.withColumn('session_id_list', \
                                          sessionize_log_udf('timestamp_list'))

In [13]:
agg_sessionized_df.show()

+--------------------+--------------------+--------------------+---------------+--------------------+
|          visitor_id|        request_list|      timestamp_list|timestamp_count|     session_id_list|
+--------------------+--------------------+--------------------+---------------+--------------------+
|1.186.41.3:1650_M...|[GET https://payt...|[2015-07-22 16:15...|              5|     [1, 1, 1, 1, 1]|
|1.186.41.45:50142...|[GET https://payt...|[2015-07-22 16:47...|              1|                 [1]|
|1.186.79.11:50014...|[GET https://payt...|[2015-07-22 16:12...|              8|[1, 1, 1, 1, 1, 1...|
|1.187.123.63:6076...|[GET https://payt...|[2015-07-22 07:10...|              1|                 [1]|
|1.187.163.231:511...|[GET https://payt...|[2015-07-22 09:05...|              1|                 [1]|
|1.187.209.36:5110...|[GET https://payt...|[2015-07-22 09:03...|             22|[1, 1, 1, 1, 1, 1...|
|1.187.210.163:212...|[GET https://payt...|[2015-07-22 16:55...|              1|  

In [14]:
# exploding aggregated data
sessionized_logs_df = (agg_sessionized_df
                            .withColumn("tmp", arrays_zip("request_list", \
                                                          "timestamp_list", \
                                                          "session_id_list"))
                            .withColumn("tmp", explode("tmp"))
                            .select("visitor_id", \
                                    "timestamp_count", \
                                    col("tmp.request_list"), \
                                    col("tmp.timestamp_list"), \
                                    col("tmp.session_id_list")))

# rename columns
sessionized_logs_df = sessionized_logs_df  \
                            .withColumnRenamed("request_list", "request")  \
                            .withColumnRenamed("timestamp_list", "timestamp") \
                            .withColumnRenamed("session_id_list", "session_id")

sessionized_logs_df.show(truncate=False)
sessionized_logs_df.createOrReplaceTempView("sessionized_logs_df")

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------+---------------------------------------------------------------------------------------------------------------------------------+-----------------------+----------+
|visitor_id                                                                                                                                                               |timestamp_count|request                                                                                                                          |timestamp              |session_id|
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------+-----------------------------------------------------------------------------------------------------------

# Validating Algorithm

Checking session_id (number of sessions)
 for visitor_id = `'119.81.61.166:55875_Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_3) AppleWebKit/534.55.3 (KHTML, like Gecko) Version/5.1.3 Safari/534.53.10'`

SESSION_THRESHOLD = 15 Minutes

 Please note - 

1.   After an inactivity of 15 minutes or more, currently active session ends
2.   Any activity recorded after a period of inactivity is considered to be a new session

In [15]:
# visitor_id = '119.81.61.166:55875_Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_3) AppleWebKit/534.55.3 (KHTML, like Gecko) Version/5.1.3 Safari/534.53.10'

# using unprocessed log data
non_sessionized_df = spark.sql("""
                      SELECT timestamp 
                      FROM derived_df 
                      WHERE visitor_id = '119.81.61.166:55875_Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_3) AppleWebKit/534.55.3 (KHTML, like Gecko) Version/5.1.3 Safari/534.53.10'
                      """)

non_sessionized_df.show(truncate=False)

# using processed (sessionized) log data
sessionized_df = spark.sql("""
                      SELECT timestamp, session_id 
                      FROM sessionized_logs_df 
                      WHERE visitor_id = '119.81.61.166:55875_Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_3) AppleWebKit/534.55.3 (KHTML, like Gecko) Version/5.1.3 Safari/534.53.10'
                      """)

sessionized_df.show(truncate=False)

+-----------------------+
|timestamp              |
+-----------------------+
|2015-07-22 07:05:40.339|
|2015-07-22 10:43:28.199|
|2015-07-22 16:17:06.159|
|2015-07-22 16:35:41.167|
|2015-07-22 16:53:29.416|
|2015-07-22 21:22:34.188|
+-----------------------+

+-----------------------+----------+
|timestamp              |session_id|
+-----------------------+----------+
|2015-07-22 07:05:40.339|1         |
|2015-07-22 10:43:28.199|2         |
|2015-07-22 16:17:06.159|3         |
|2015-07-22 16:35:41.167|4         |
|2015-07-22 16:53:29.416|5         |
|2015-07-22 21:22:34.188|6         |
+-----------------------+----------+



## **2. Determine the average session time**

* Average Session Time = Total Session Duration / Total Number of Sessions



In [16]:
total_number_of_sessions_df = spark.sql("""
              SELECT SUM(number_of_session_per_visitor) total_number_of_sessions 
              FROM 
                (
                  SELECT visitor_id, 
                  MAX(session_id) number_of_session_per_visitor
                  FROM sessionized_logs_df 
                  GROUP BY visitor_id
                )
              """)

total_number_of_sessions_df.show()
total_number_of_sessions_list = total_number_of_sessions_df \
                                      .select('*').collect()
total_number_of_sessions = total_number_of_sessions_list[0] \
                                      .total_number_of_sessions

+------------------------+
|total_number_of_sessions|
+------------------------+
|                  436525|
+------------------------+



In [17]:
total_session_duration_df = spark.sql("""
              SELECT SUM(session_duration) total_session_duration
              FROM
                (
                  SELECT visitor_id, session_id,
                  MAX(timestamp) max_timestamp,
                  MIN(timestamp) min_timestamp,                
                  UNIX_TIMESTAMP(MAX(timestamp)) - UNIX_TIMESTAMP(MIN(timestamp))
                  session_duration
                  FROM sessionized_logs_df
                  GROUP BY visitor_id, session_id
                )
              """)

total_session_duration_df.show()
total_session_duration_list = total_session_duration_df.select('*').collect()
total_session_duration = total_session_duration_list[0].total_session_duration

+----------------------+
|total_session_duration|
+----------------------+
|             310072597|
+----------------------+



In [18]:
# average session time
average_session_time = total_session_duration / total_number_of_sessions

print("Average Session Time / A session on average lasted -\n")
print("%s Seconds\n" % average_session_time)

print("(When the timeout threshold was set to 15 Minutes = 900 Seconds)")

Average Session Time / A session on average lasted -

710 Seconds

(When the timeout threshold was set to 15 Minutes = 900 Seconds)


## **3. Determine unique URL visits per session. To clarify, count a hit to a unique URL only once per session**

**Most number of hits in a session were seen for following URL:**


*   GET https://paytm.com:443/bus-tickets/search/Delhi/Katra/2015-07-31/1 HTTP/1.1 - 1946 hits






In [19]:
# unique URL visits per session *per visitor*

unique_url_visit_per_session = spark.sql("""
                  SELECT visitor_id, session_id, request, 
                  COUNT(request) number_of_request
                  FROM sessionized_logs_df
                  GROUP BY visitor_id, session_id, request
                  ORDER BY number_of_request DESC
                  """)

unique_url_visit_per_session.show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------+----------+------------------------------------------------------------------------------+-----------------+
|visitor_id                                                                                                                            |session_id|request                                                                       |number_of_request|
+--------------------------------------------------------------------------------------------------------------------------------------+----------+------------------------------------------------------------------------------+-----------------+
|112.196.25.164:55986_Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.2.28) Gecko/20120313 Thunderbird/3.1.20                           |1         |GET https://paytm.com:443/bus-tickets/search/Delhi/Katra/2015-07-31/1 HTTP/1.1|1946             |
|112.196.25.164:4279

## **4. Find the most engaged users, ie the IPs with the longest session times**

**Most engaged user with the longest session times was:**


- **Visitor / User ID -** 119.81.61.166:36205_Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_3) AppleWebKit/534.55.3 (KHTML, like Gecko) Version/5.1.3 Safari/534.53.10

- **Session Duration -** 67414 seconds







In [20]:
# longest session times
most_engaged_users = spark.sql("""
                  SELECT visitor_id, session_id,
                  MAX(timestamp) max_timestamp,
                  MIN(timestamp) min_timestamp,                
                  UNIX_TIMESTAMP(MAX(timestamp)) - UNIX_TIMESTAMP(MIN(timestamp))
                  session_duration
                  FROM sessionized_logs_df
                  GROUP BY visitor_id, session_id
                  ORDER BY session_duration DESC
              """)

most_engaged_users.show(truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------+----------+-----------------------+-----------------------+----------------+
|visitor_id                                                                                                                                |session_id|max_timestamp          |min_timestamp          |session_duration|
+------------------------------------------------------------------------------------------------------------------------------------------+----------+-----------------------+-----------------------+----------------+
|119.81.61.166:36205_Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_3) AppleWebKit/534.55.3 (KHTML, like Gecko) Version/5.1.3 Safari/534.53.10|1         |2015-07-22 21:25:43.163|2015-07-22 02:42:09.879|67414           |
|106.186.23.95:42887_-                                                                                                              